In [1]:
!pip uninstall -y torch # Uninstall current torch version to avoid potential conflicts
!pip install torch==2.0.1+cu118 --index-url https://download.pytorch.org/whl/cu118  # Reinstall specific torch version, specify CUDA version if needed
!pip install vllm transformers==4.31.0

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 805.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=5976f8b5925e82708943bd639771bcde8c26c6bd6abe07b4e8cd3901c260ccba
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [1]:
import vllm
import json
import os
from transformers import AutoModelForSeq2SeqLM, AutoConfig, AutoTokenizer

# Create local model directory if it doesn't exist
model_dir = "model_cache"
os.makedirs(model_dir, exist_ok=True)

# Specify the Hugging Face model identifier
model_name = "google/flan-t5-xl"  # Example model, can be replaced with any compatible model

# Download and cache the model configuration and weights
config = AutoConfig.from_pretrained(model_name, cache_dir=model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=config, cache_dir=model_dir)

# Download and cache the tokenizer (if needed)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=model_dir)

# Set the model path to the cached directory + config file
model_path = os.path.join(model_dir, "config.json")

# --- vLLM Integration with Hugging Face Model ---
from vllm import LLM, SamplingParams

# Initialize vLLM with a supported model (e.g., Mistral or any other LLM model)
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1")

# --- Turing Tumble Hybrid Neuro-Symbolic AI ---
# Example board JSON structure
test_board_json = {
    "blue_bits": 3,
    "red_bits": 2,
    "goal": "drop a blue ball into the rightmost bin"
}

# --- Generate LLM Prompt Function ---
def generate_llm_prompt(board_json):
    # Extract parts of the board JSON
    blue_bits = board_json.get("blue_bits", 0)
    red_bits = board_json.get("red_bits", 0)
    goal = board_json.get("goal", "solve the board")

    # Build a structured prompt for LLM
    prompt = (
        f"You are an expert in solving Turing Tumble puzzles.\n"
        f"The board has {blue_bits} blue bits and {red_bits} red bits installed.\n"
        f"Your task is to plan a sequence of logical steps or component placements "
        f"to achieve the goal: '{goal}'.\n"
        f"Return your plan as a list of steps in JSON format, like:\n"
        f'["Place a gear at (3,2)", "Set bit 1 to ON", "Connect ramp from (2,3) to (1,4)", ...]\n'
    )

    return prompt

# --- Modified plan_with_llm function ---
def plan_with_llm(board_json):
    # Generate the prompt based on board state
    prompt = generate_llm_prompt(board_json)

    # Define generation parameters for LLM
    sampling_params = SamplingParams(temperature=0.7, max_tokens=128)

    # Generate response using vLLM
    outputs = llm.generate([prompt], sampling_params=sampling_params)

    # Extract and process the generated text
    response_text = outputs[0].outputs[0].text

    # Assuming the response is JSON, parse it
    try:
        return json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: LLM response is not valid JSON. Returning empty action.")
        return {}  # Return an empty action if JSON parsing fails

# Example usage:
generated_plan = plan_with_llm(test_board_json)

# Print the generated plan
print(json.dumps(generated_plan, indent=4))

# --- Run Hybrid Pipeline ---
# Here you would integrate the rest of your pipeline, such as encoding the generated plan,
# interpreting the results, and running the hybrid AI steps (e.g., image recognition, GNN reasoning).




INFO 05-05 11:14:36 [__init__.py:239] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO 05-05 11:15:07 [config.py:717] This model supports multiple tasks: {'generate', 'score', 'classify', 'embed', 'reward'}. Defaulting to 'generate'.
WARNING 05-05 11:15:07 [arg_utils.py:1658] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 05-05 11:15:07 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='mistralai/Mistral-7B-Instruct-v0.1', speculative_config=None, tokenizer='mistralai/Mistral-7B-Instruct-v0.1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=Obser

/usr/local/lib/python3.11/dist-packages/vllm/transformers_utils/tokenizer_group.py:23: FutureWarning: It is strongly recommended to run mistral models with `--tokenizer-mode "mistral"` to ensure correct encoding and decoding.
  self.tokenizer = get_tokenizer(self.tokenizer_id, **tokenizer_config)


INFO 05-05 11:15:09 [cuda.py:240] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 05-05 11:15:09 [cuda.py:289] Using XFormers backend.


ValueError: Bfloat16 is only supported on GPUs with compute capability of at least 8.0. Your Tesla T4 GPU has compute capability 7.5. You can use float16 instead by explicitly setting the `dtype` flag in CLI, for example: --dtype=half.

In [ ]:
# Turing Tumble Hybrid Neuro-Symbolic AI (Colab Notebook Version)

# ------------------------
# Install Dependencies (Colab only)
# ------------------------
!pip install -q transformers torch torchvision torch-geometric networkx openai Pillow
!pip install -q openai
# ------------------------
# Imports
# ------------------------
import torch
from torchvision import transforms
from transformers import BlipProcessor, BlipForConditionalGeneration
from torch_geometric.data import Data as GNNData
from torch_geometric.nn import GCNConv
import networkx as nx
from PIL import Image
import json
import openai
import matplotlib.pyplot as plt
import os # Import the os module

from openai import OpenAI
client = OpenAI()

# Set your OpenAI API key using an environment variable
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"  # Replace YOUR_API_KEY with your actual API key


response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

# ------------------------
# Upload Image (Colab)
# ------------------------
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# ------------------------
# VLM Caption Extraction (BLIP)
# ------------------------
def extract_board_description_with_blip(image_path):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

# ------------------------
# Dummy Logic Graph Construction
# ------------------------
def build_dummy_graph_from_caption(caption):
    G = nx.DiGraph()
    G.add_node(0, label="ball", position=[0, 0])
    G.add_node(1, label="ramp", position=[1, 0], orientation="right")
    G.add_node(2, label="gear", position=[2, 1])
    G.add_edges_from([(0, 1), (1, 2)])
    return G

# ------------------------
# Convert to PyTorch Geometric Format
# ------------------------
def convert_nx_to_gnn_data(graph):
    node_labels = [ord(graph.nodes[n]['label'][0]) for n in graph.nodes()]
    x = torch.tensor([[l] for l in node_labels], dtype=torch.float)
    edge_index = torch.tensor(list(graph.edges())).t().contiguous()
    return GNNData(x=x, edge_index=edge_index)

# ------------------------
# Serialize for LLM
# ------------------------
def serialize_board_to_json(graph):
    nodes = []
    for nid, attrs in graph.nodes(data=True):
        nodes.append({
            "id": nid,
            "type": attrs.get("label", "unknown"),
            "position": attrs.get("position", [0, 0]),
            "orientation": attrs.get("orientation", "none")
        })
    edges = [{"from": u, "to": v} for u, v in graph.edges()]
    return {
        "nodes": nodes,
        "connections": edges,
        "marbles": [],
        "gear_states": [],
        "bit_states": [],
        "goal": "Trigger final bit to ON using blue marble"
    }

# ------------------------
# LLM Planning Prompt
# ------------------------
def generate_llm_prompt(board_json):
    return f"""
You are an AI agent acting as a symbolic planner for a Turing Tumble puzzle.
Your goal is: \"{board_json['goal']}\"

Here is the current board state:
{json.dumps(board_json, indent=2)}

Reply with the best next action using this format:
{{
  \"action\": \"place_component\",
  \"component\": \"ramp\",
  \"position\": [4, 5],
  \"orientation\": \"left\"
}}

Only use these actions:
- \"place_component\"
- \"remove_component\"
- \"launch_marble\"
"""

def plan_with_llm(board_json):
    prompt = generate_llm_prompt(board_json)
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return json.loads(response["choices"][0]["message"]["content"])

# ------------------------
# Simple GNN Model
# ------------------------
class TuringTumbleGNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# ------------------------
# Pipeline
# ------------------------
def hybrid_pipeline(image_path):
    print("[1] Extracting board info with VLM...")
    caption = extract_board_description_with_blip(image_path)
    print("\n[Caption]:", caption)

    print("[2] Building logic graph from caption...")
    nx_graph = build_dummy_graph_from_caption(caption)

    print("[3] Serializing for LLM planner...")
    board_json = serialize_board_to_json(nx_graph)

    print("[4] Invoking LLM planner...")
    action = plan_with_llm(board_json)
    print("\n[LLM Suggested Action]:", action)

    print("[5] Symbolic reasoning with GNN...")
    gnn_data = convert_nx_to_gnn_data(nx_graph)
    model = TuringTumbleGNN()
    output = model(gnn_data)
    print("\n[GNN Output]:", output)

    return output, action

# ------------------------
# Run Pipeline
# ------------------------
output, action = hybrid_pipeline(image_path)